In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import multiprocessing
import matplotlib.pyplot as plt

In [5]:
class data_proc():
    def process_zip():
        #dealing with zipcode
        zip_data=pd.read_csv('ZIP.csv')
        zip_data['Zip']=zip_data['Zip'].astype(str)
        for i in range(len(zip_data['Zip'])):
            if len(zip_data['Zip'][i])==4:
                zip_data['Zip'][i]='0'+zip_data['Zip'][i]
        zip_data['Zip']=[zip_data['Zip'][i][0:3] for i in range(len(zip_data))] 
        zip_data.iloc[:,-3:]=zip_data[['Median','Mean','Pop']].apply(lambda x: x.str.replace(',',''))
        for i in range(1,4):
            zip_data.iloc[:,-i]=pd.to_numeric(zip_data.iloc[:,-i],errors='coerce')
        zip_data['weight']=zip_data['Pop']/zip_data.groupby('Zip')['Pop'].transform(sum)
        zip_data['new_mean']=zip_data['Mean']*zip_data['weight']
        zip_data['new_median']=zip_data['Median']*zip_data['weight']
        zip_new=pd.DataFrame()
        zip_new=zip_data.groupby('Zip')['new_mean','new_median'].sum()
        return zip_new
        
    def readcsv():
        LARGE_FILE = "C:\Users\Administrator\Desktop\practicum_regression\loan_data_no_current_converted.csv"
        CHUNKSIZE = 100000 # processing 100,000 rows at a time
        reader = pd.read_csv(LARGE_FILE, chunksize=CHUNKSIZE, low_memory=False)
        frames = []
        for df in reader:
            frames.append(df)
        loan_data = pd.concat(frames)
        return loan_data    
        
    def cleaning(df,zip_new,keep_desc=True,categorical_to_binary=True):
        #drop the observation that was missing for ALL field
        df=df.dropna(axis=0,how='all')
        #drop the meaningless features
        drop_list=['emp_title','title','earliest_cr_line','desc','issue_d','id','member_id','url','grade','sub_grade',
                   'int_rate','avg_cur_bal','addr_state','funded_amnt','funded_amnt_inv','collection_recovery_fee',
                   'collections_12_mths_ex_med','mths_since_last_major_derog','next_pymnt_d','recoveries','total_pymnt',
                   'total_pymnt_inv','total_rec_int','issue_d',' last_credit_pull_d','last_pymnt_d','last_credit_pull_d']
        df.drop(drop_list,inplace=True,axis=1,errors='ignore')
        
        #deal with percentage mark
        df['revol_util']=df['revol_util'].replace('%','',regex=True).astype('float')/100
        
        #dealing with categorical features
        if categorical_to_binary==True:
            categorical_features=['addr_state','application_type','emp_length','grade','home_ownership','initial_list_status','pymnt_plan','sub_grade','term','verification_status']
            for i in categorical_features:
                if i in list(df):
                    df[i]=df[i].astype('category')
                    df=pd.get_dummies(df,columns={i},drop_first=True)
        
        #merge zipcode with census data
        df['zip_code']=df['zip_code'].apply(lambda x: x[:3])
        df=df.join(zip_new,on='zip_code')
        df.drop('zip_code',inplace=True,axis=1)
        
        #drop the features for which greater than 10% of the loans were missing data for
        num_rows=df.count(axis=0)
        df=df.iloc[:,(num_rows>=0.9*len(df)).tolist()]
        #drop the observation that was missing for any field
        df=df.dropna(axis=0,how='any')
       
        
        #label the dataset to create y
        y=df['loan_status']
        df=df.drop(['loan_status'],axis=1)    
        return df,y
    
    '''
    The following part is updated by Yufei Gao on 3/29
    The main purpose is to deal with the imbalance dataset
    
    '''
    #we firstly simplify this problem to use only for categories:
    #fully paid, grace period, late and charged off
    
    def simplify_status(loan_data):
        #does not meet policy: fully paid, can be also considered as fully paid
        loan_data['loan_status'].replace(2,1,inplace = True)
        #does not meet policy: charged off, can be also considered as charged off
        loan_data['loan_status'].replace(3,8,inplace = True)
        #Default merged to charged off
        loan_data['loan_status'].replace(7,8,inplace = True)
        #merge those "late" status
        loan_data['loan_status'].replace(6,5,inplace = True)
        
        #By doing these, we have 1-fully paid, 4-grace period
        # 5-late, and 8-charged off
        #Then we can renumber these categories, as:
        #1-fully paid, 2-grace period, 3-late, 4-charged off
        loan_data['loan_status'].replace(4,2,inplace = True)
        loan_data['loan_status'].replace(8,4,inplace = True)
        loan_data['loan_status'].replace(5,3,inplace = True)
        return loan_data
                
            
    
    #calculate the distribution in the column: loan_status
    def calculate_status(df):
        dic1={}
        for val in df['loan_status']:
            if val in dic1:
                dic1[val] += 1;
            else:
                dic1[val] = 1;
        return dic1
    
    #one method to deal with the imbalance dataset is to cut some samples
    #which belong to the major categories
    #input variables: the dataset, calculation based on each status
    #and remain number for the category 1
    
    def imbalance_cut(df, dic, remain_perc):
        #seperate the df based on loan status
        #notice that the number of 1 and 4 is too large
        df1 = df[df['loan_status'] == 1]
        df2 = df[df['loan_status'] == 2]
        df3 = df[df['loan_status'] == 3]
        df4 = df[df['loan_status'] == 4]
        
        remain_num1 = int(remain_perc * dic[1])
        remain_num4 = remain_num1
        #split those 2 categories randomly
        df1= df1.sample(remain_num1, replace=True)
        df4= df4.sample(remain_num4, replace=True)
        
        frames = []
        #concate all left
        frames.append(df1)
        frames.append(df2)
        frames.append(df3)
        frames.append(df4)
        loan_data = pd.concat(frames)
        return loan_data
    
    #the other method is to duplicate the category with relatively small total numbers
    def imbalance_dup(df, dic, dup_times):
        #seperate the df based on loan status
        #notice that the number of 1 and 4 is too large
        #other categories are relativly small size
        df1 = df[df['loan_status'] == 1]
        df2 = df[df['loan_status'] == 2]
        df3 = df[df['loan_status'] == 3]
        df4 = df[df['loan_status'] == 4]
        
        #duplicate those categories for several times
        #try to make sure their size are closed to each other
        frames = []
        for times in range(dup_times):          
            frames.append(df2)
        for times in range(int(dup_times*dic[2]/dic[3])):
            frames.append(df3)            
        
        frames.append(df1)
        frames.append(df4)
        loan_data = pd.concat(frames)
        return loan_data
    
    #the third method is to combine cut and duplication
    #that is, we cut the size of cate 1 and 8 while duplicate other categories
    def imblance_cut_dup(self, df, dic, remain_perc, dup_times):
        loan_data = self.imbalance_cut(df, dic, remain_perc)
        loan_data = self.imbalance_dup(loan_data, dic, dup_times)
        
        dic1={}
        for val in loan_data['loan_status']:
            if val in dic1:
                dic1[val] += 1;
            else:
                dic1[val] = 1;
        print "the current distribution:"
        print dic1
        return loan_data

### Below is a test for these functions

In [3]:
LARGE_FILE = "C:\Users\Administrator\Desktop\practicum_regression\loan_data_no_current_converted.csv"
CHUNKSIZE = 100000 # processing 100,000 rows at a time
reader = pd.read_csv(LARGE_FILE, chunksize=CHUNKSIZE, low_memory=False)
frames = []
for df in reader:
    frames.append(df)
loan_data = pd.concat(frames)

In [4]:
def simplify_status(loan_data):
    loan_data['loan_status'].replace(2,1,inplace = True)
    loan_data['loan_status'].replace(3,8,inplace = True)
    loan_data['loan_status'].replace(7,8,inplace = True)
    loan_data['loan_status'].replace(6,5,inplace = True)
    loan_data['loan_status'].replace(4,2,inplace = True)
    loan_data['loan_status'].replace(8,4,inplace = True)
    loan_data['loan_status'].replace(5,3,inplace = True)
    return loan_data

In [6]:
def calculate_status(df):
        dic1={}
        for val in df['loan_status']:
            if val in dic1:
                dic1[val] += 1;
            else:
                dic1[val] = 1;
        return dic1

In [5]:
    def imbalance_cut(df, dic, remain_perc):
        #seperate the df based on loan status
        #notice that the number of 1 and 4 is too large
        df1 = df[df['loan_status'] == 1]
        df2 = df[df['loan_status'] == 2]
        df3 = df[df['loan_status'] == 3]
        df4 = df[df['loan_status'] == 4]
        
        remain_num1 = int(remain_perc * dic[1])
        remain_num4 = remain_num1
        #split those 2 categories randomly
        df1= df1.sample(remain_num1, replace=True)
        df4= df4.sample(remain_num4, replace=True)
        
        frames = []
        #concate all left
        frames.append(df1)
        frames.append(df2)
        frames.append(df3)
        frames.append(df4)
        loan_data = pd.concat(frames)
        return loan_data
    
    #the other method is to duplicate the category with relatively small total numbers
    def imbalance_dup(df, dic, dup_times):
        #seperate the df based on loan status
        #notice that the number of 1 and 4 is too large
        #other categories are relativly small size
        df1 = df[df['loan_status'] == 1]
        df2 = df[df['loan_status'] == 2]
        df3 = df[df['loan_status'] == 3]
        df4 = df[df['loan_status'] == 4]
        
        #duplicate those categories for several times
        #try to make sure their size are closed to each other
        frames = []
        for times in range(dup_times):          
            frames.append(df2)
        for times in range(int(dup_times*dic[2]/dic[3])):
            frames.append(df3)            
        
        frames.append(df1)
        frames.append(df4)
        loan_data = pd.concat(frames)
        return loan_data
    
    #the third method is to combine cut and duplication
    #that is, we cut the size of cate 1 and 8 while duplicate other categories
    def imblance_cut_dup(df, dic, remain_perc, dup_times):
        loan_data = imbalance_cut(df, dic, remain_perc)
        loan_data = imbalance_dup(loan_data, dic, dup_times)
        
        dic1={}
        for val in loan_data['loan_status']:
            if val in dic1:
                dic1[val] += 1;
            else:
                dic1[val] = 1;
        print "the current distribution:"
        print dic1
        return loan_data

In [8]:
dic = calculate_status(loan_data)
print dic
print "new fully paid:",dic[1]+dic[2]
print "new charged off:",dic[7]+dic[3]+dic[8]
print "grace:",dic[4]
print "late:",dic[5]+dic[6]

{1: 673688, 2: 1988, 3: 761, 4: 12125, 5: 5605, 6: 22452, 7: 119, 8: 175085}
new fully paid: 675676
new charged off: 175965
grace: 12125
late: 28057


### use the function "simplify_status" to merge some status

In [9]:
loan_data_copy = loan_data.copy()
loan_data_copy = simplify_status(loan_data_copy)

In [10]:
dic1 = calculate_status(loan_data_copy)
print dic1

{1: 675676, 2: 12125, 3: 28057, 4: 175965}


### We can find that the number of each status perfectly matches

In [11]:
loan_data_copy = imblance_cut_dup(loan_data_copy, dic1, 0.1, 6)
#remain 10% of the status 1, and duplicate type 2 for 6 times
# dict1 is definded above

the current distribution:
{1: 67567, 2: 72750, 3: 56114, 4: 67567}


### The new distribution of the 4 status